In [1]:
from coffea.nanoevents import NanoEventsFactory, BaseSchema, PFNanoAODSchema
import json
import fastjet
import numpy as np
import awkward as ak
from coffea import processor
import hist
import coffea.nanoevents.methods.vector as vector
import warnings
import matplotlib.pyplot as plt
import math

/tmp/ipykernel_1036804/1518451047.py:8: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  import coffea.nanoevents.methods.vector as vector


In [2]:
warnings.filterwarnings("ignore", module="coffea.*")
warnings.filterwarnings("ignore", "Found duplicate branch")
warnings.filterwarnings("ignore", "Missing cross-reference index for")
warnings.filterwarnings("ignore", "dcut")
warnings.filterwarnings("ignore", "Please ensure")
warnings.filterwarnings("ignore", "invalid value")

In [3]:
with open('../../filelists/hgg_files.txt', 'r') as f:
    hgg_files = [line.strip() for line in f]

In [4]:
hgg = NanoEventsFactory.from_root(
    {'/project01/ndcms/cmoore24/signal/hgg/' + hgg_files[0]: "/Events"},
    delayed=False,
    schemaclass=PFNanoAODSchema,
    metadata={"dataset": "Hgg"},
).events()

In [5]:
fatjet = hgg.FatJet

In [6]:
pf = ak.flatten(fatjet.constituents.pf, axis=1)

In [7]:
jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 0.2)

In [8]:
cluster = fastjet.ClusterSequence(pf, jetdef)

#--------------------------------------------------------------------------
#                         FastJet release 3.4.1
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the GNU GPL v2 or higher.  
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code,
# CGAL and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------


In [9]:
subjets = cluster.inclusive_jets()

In [189]:
vec = ak.zip(
    {
        "x": subjets.px,
        "y": subjets.py,
        "z": subjets.pz,
        "t": subjets.E,
    },
    with_name="LorentzVector",
    behavior=vector.behavior,
)

In [221]:
vec = ak.pad_none(vec, 3)

In [222]:
vec["idx"] = ak.local_index(vec)

In [223]:
i, j = ak.unzip(ak.combinations(vec, 2))

In [224]:
best = ak.argmax(abs((i + j).pt), axis=1, keepdims=True)

In [225]:
order_check = ak.concatenate([i[best].pt, j[best].pt], axis=1)

In [226]:
leading = ak.argmax(order_check, axis=1, keepdims=True)

In [227]:
subleading = ak.argmin(order_check, axis=1, keepdims=True)

In [228]:
leading_particles = ak.concatenate([i[best], j[best]], axis=1)

In [229]:
cut = ((vec.idx != ak.firsts(leading_particles.idx)) & 
       (vec.idx != ak.firsts(ak.sort(leading_particles.idx, ascending=False)))
      )

In [230]:
everything_else = vec[cut]

In [231]:
total_soft_pt = ak.sum(everything_else.pt, axis=1)

In [232]:
everything_else['momentum_fraction'] = (everything_else.pt)/total_soft_pt

In [233]:
everything_else['weighted_eta'] = everything_else.eta * everything_else.momentum_fraction

In [234]:
everything_else['weighted_phi'] = everything_else.phi * everything_else.momentum_fraction

In [235]:
weighted_average_eta = ak.sum(everything_else.weighted_eta, axis=1)/ak.num(everything_else, axis=1)

In [236]:
average_eta = ak.sum(everything_else.eta, axis=1)/ak.num(everything_else, axis=1)

In [237]:
weighted_average_phi = ak.sum(everything_else.weighted_phi, axis=1)/ak.num(everything_else, axis=1)

In [238]:
average_phi = ak.sum(everything_else.phi, axis=1)/ak.num(everything_else, axis=1)

In [239]:
leg1 = leading_particles[leading]
leg2 = leading_particles[subleading]

In [256]:
a13 = ak.flatten((((leg1.eta - average_eta)**2) + 
       ((leg1.phi - average_phi)**2)
      ))
a23 = ak.flatten((((leg2.eta - average_eta)**2) + 
       ((leg2.phi - average_phi)**2)
      ))
a12 = ak.flatten(leg1.delta_r2(leg2))

In [241]:
color_ring = (a13 + a23) / (a12)

In [264]:
leading_particles[0]

<LorentzVectorArray [{x: 14.5, y: -29.6, z: 18.8, ...}, ...] type='2 * ?Lor...'>

In [267]:
leg1[0]

<LorentzVectorArray [{x: 104, y: -153, z: 233, t: 297, ...}] type='1 * ?Lor...'>